
# Import Python Packages


In [1]:
# from tensorflow_core.python.keras.callbacks import ModelCheckpoint
# from tensorflow_core.python.keras.utils.data_utils import Sequence
# from tensorflow_core.python.keras.utils.layer_utils import print_summary
# from tensorflow_core.python.keras.utils.np_utils import to_categorical
from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import Sequence, to_categorical
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model

print("yolo")
import os

import glob
# from mtcnn.mtcnn import MTCNN
import re

import numpy as np
import shutil
from resnet_convlstm import ResNet
# from src.xception_convlstm import Xception
# print(cv2.__version__)
from PIL import Image
import random
from sklearn.utils import class_weight
from datetime import datetime as dt
import natsort
import matplotlib.pyplot as plt
import cv2
random.seed(32)
dataset_dir='DeepFakeDatasetReal'
# from src.cl_basic import cl_basic
import ipykernel
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="2"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)


# DFVDSequence Class

In [2]:
#   Here, `x_set` is list of path to the images,
#   `y_set` are the associated classes,
#   'v' is the no. of videos for each batch
#   and fpv is the no. of frames from each video
class DFVDSequence(Sequence):
    def __init__(self, x_set, y_set,v,fpv,image_size=240):
        self.x, self.y = x_set, y_set
        self.batch_size = fpv*v
        self.count=0
        self.video_iterator=0
        self.frame_iterator=0
        self.v=v
        self.fpv=fpv
        self.dataset_size=sum([len(value) for key, value in self.x.items()])
        self.frame_counter=0
        self.on_epoch_end()
        self.image_size=image_size
    def __len__(self):
        return int(np.ceil(self.dataset_size / float(self.batch_size)))
    def resize_img(self,img,basewidth=240):
        wpercent = (basewidth/float(img.size[0]))
        hsize = int((float(img.size[1])*float(wpercent)))
#         img = img.resize((basewidth,hsize), Image.ANTIALIAS)
        img = img.resize((basewidth,basewidth), Image.ANTIALIAS)
        return img
    
    def img_transf(self,img):
        img -= img.mean()
        img /= np.maximum(img.std(), 1/image_size**2) #prevent /0
        return img  
        
    def __getitem__(self,idx):
        batch_x=[]
        batch_y = []
        video_idx=0
        _iter=0
        while _iter < self.v:
            _iter+=1
            if self.frame_counter >= self.dataset_size:
                self.on_epoch_end()
                print('*',end=",")
                _iter-=1
                continue
            frames2read=self.x[str(self.video_iterator)][self.frame_iterator:self.frame_iterator+self.fpv]
            temp_frames=[]
            img_gen = ImageDataGenerator(rotation_range=30,
                                         samplewise_center=True,
                                         samplewise_std_normalization=True,
                                        # width_shift_range=0.2,
                                        # height_shift_range=0.2,
                                        # rescale=1./255,
                                        brightness_range=[0.7,1.0],
                                        channel_shift_range=50.0,
                                        # shear_range=0.1,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
#                                         vertical_flip = True,
                                        fill_mode='nearest')
            transform_param=img_gen.get_random_transform(img_shape=(self.image_size,self.image_size,3), seed=None)
#             print(transform_param)
            if len(frames2read)>=self.fpv:
                for frame in frames2read:                    
                    _image=cv2.imread(frame)
                    _image=cv2.cvtColor(_image, cv2.COLOR_BGR2RGB).astype('float64')
                    _image=cv2.resize(_image,(self.image_size,self.image_size))
                    _image=img_gen.apply_transform(_image,transform_param)
                    _image/=255
                    temp_frames.append(np.asarray(_image))
                batch_x.append(temp_frames)
                batch_y.append(self.y[self.video_iterator])
            else:
                _iter-=1
            self.video_iterator+=1
            self.frame_counter+=len(frames2read)
            if self.video_iterator % len(self.x) ==0:
                self.video_iterator=0
                self.frame_iterator+=len(frames2read)
            video_idx+=1
        if self.video_iterator % len(self.x) == 0:
            self.video_iterator=0
        if len(batch_y)>0:
            batch_y=to_categorical(batch_y,2)
            batch_x=np.array(batch_x)
        return (batch_x,batch_y)
    def on_epoch_end(self):
        """ Method called at the end of every epoch. """
        self.count=0
        self.video_iterator=0
        self.frame_iterator=0
        self.frame_counter=0
        return


# Sequence Loader

In [3]:

def downsample_withoutshuffle(f_list,fpvpb,no_frames):
    f_list=natsort.natsorted(f_list)
    _total=int(no_frames/fpvpb)# 20/5
    _tempList=[]
    # print("Size:",_total)
    if len(f_list)>_total:#if list smaller than required frames
        for i in range (0,_total):
            _range=len(f_list)-fpvpb
            if _range==0:
                return f_list
            x=random.randrange(_range)
            for j in range(0, fpvpb):
                # print(os.path.splitext(os.path.basename(f_list[x+j]))[0],end=",")
                _tempList.append(f_list[x+j])
            # print()
        return _tempList
    else:
        return f_list
    
def create_sequence(dir,fpvpb,no_frames):
    random.seed(35)
    count_real=0
    count_fake=0
    folders = [i for i in sorted(glob.glob(os.path.join(dir,'*',"*")))]
    total_folders=len(folders)
    X={}
    y = []
    print('Total Video Folders Found (Real + Fake):',total_folders)
    pre_folder=-1
    i=0
    _downsampled=[]
    while i<total_folders:
        folder=random.choice(folders)
        dir_name=os.path.dirname(folder)
        folder_name=os.path.basename(dir_name)
#         print(folder_name)
        _downsampled=[x for x in sorted(glob.glob(os.path.join(folder,"*")))]
#         _downsampled=downsample_withoutshuffle([x for x in sorted(glob.glob(os.path.join(folder,"*")))],fpvpb,no_frames)
        if folder_name=='real':
            if pre_folder == 1:
                continue
            y.append(1)
            count_real+=len(_downsampled)
            pre_folder = 1
        elif folder_name=='fake':
            if pre_folder == 0:
                continue
            y.append(0)
            count_fake+=len(_downsampled)
            pre_folder = 0
        else:
            print("Directory names should be 'real' for label (1) and 'fake' for label (0)")
            exit(0)
        # X[str(i)]=[x for x in sorted(glob.glob(os.path.join(folder,"*")))]
        X[str(i)]=_downsampled
        folders.remove(folder)
        i+=1
    print('Real Frames:',count_real,'Fake Frames:',count_fake)
    labels=[]
    for i in range(0,count_real):
        labels.append(1)
    for i in range(0,count_fake):
        labels.append(0)
    y_ints = [v.argmax() for v in to_categorical(labels)]
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_ints), y_ints)
    print('Class Weights:',class_weights)
    return X,y, class_weights

train_video_per_batch=16
val_video_per_batch=16
test_video_per_batch=16
frames_per_video=80 #total frames
frames_per_video_per_batch=5 #frames in one batch
image_size=240
channel=3
X_train,y_train, class_weights_train =create_sequence('datasets/FaceSwap/train/',frames_per_video_per_batch,frames_per_video)
X_val,y_val,class_weights_val=create_sequence('datasets/FaceSwap/val/',frames_per_video_per_batch,frames_per_video)
# X_test,y_test,class_weights_test=create_sequence('Deepfakes/test/',frames_per_video_per_batch,frames_per_video)
train_it=DFVDSequence(X_train,y_train,train_video_per_batch,frames_per_video_per_batch,image_size)
val_it=DFVDSequence(X_val,y_val,val_video_per_batch,frames_per_video_per_batch,image_size)
# test_it=DFVDSequence(X_test,y_test,test_video_per_batch,frames_per_video_per_batch,image_size)

fig=plt.figure(figsize=(10, 100))
columns = 5
rows = 40

print("Training")
x=1
temp=train_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
plt.imshow(np.array(temp[0][0,0]))
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][0,1]))
fig.add_subplot(rows, columns, x+2)
plt.imshow(np.array(temp[0][0,2]))
fig.add_subplot(rows, columns, x+3)
plt.imshow(np.array(temp[0][0,3]))
fig.add_subplot(rows, columns, x+4)
plt.imshow(np.array(temp[0][0,4]))
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
fig.add_subplot(rows, columns, x+5)
plt.imshow(np.array(temp[0][3,0]))
fig.add_subplot(rows, columns, x+6)
plt.imshow(np.array(temp[0][3,1]))
fig.add_subplot(rows, columns, x+7)
plt.imshow(np.array(temp[0][3,2]))
fig.add_subplot(rows, columns, x+8)
plt.imshow(np.array(temp[0][3,3]))
fig.add_subplot(rows, columns, x+9)
plt.imshow(np.array(temp[0][3,4]))
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
print(temp[0].shape,temp[1].shape,train_it.dataset_size,train_it.frame_counter,train_it.__len__(),temp[1][0],temp[1][1])
# x+=2
plt.title("Training Set Examples")        
plt.show()
train_it.on_epoch_end()
fig=plt.figure(figsize=(10, 100))
columns = 4
rows = 23

print("Validation")
x=1
temp=val_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
plt.imshow(np.array(temp[0][0,0]),)
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][1,0]))
print(temp[0].shape,temp[1].shape,val_it.dataset_size,val_it.frame_counter,val_it.__len__(),temp[1][0],temp[1][1])
plt.title("Validation Set Examples")   
plt.show()
val_it.on_epoch_end()
                                                                                                            

# Model Compile

In [4]:


model=ResNet(input_shape=(frames_per_video_per_batch,image_size, image_size, channel), classes=2, block='bottleneck', residual_unit='v2',
           repetitions=[3, 4, 6, 3], initial_filters=64, activation='softmax', include_top=False,
           input_tensor=None, dropout=0.25, transition_dilation_rate=(1, 1),
           initial_strides=(2, 2), initial_kernel_size=(7, 7), initial_pooling='max',
           final_pooling=None, top='classification')
print_summary(model, line_length=150, positions=None, print_fn=None)


In [5]:

adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# model.compile(loss='binary_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="2"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
_id = dt.now().strftime("%y-%m-%d-%H_%M")
filepath='ResNet50(bk)_'+_id+"-{epoch:02d}-{acc:.2f}"
checkpoint = ModelCheckpoint(str(filepath)+".hdf5")#, monitor='acc', verbose=2, save_best_only=True, mode='max')
csv_logger = CSVLogger('ResNet50(bk)_'+str(_id)+".csv", append=True, separator=',')

# earlyStop=EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
callbacks_list = [checkpoint,csv_logger]


# Training

In [ ]:
model.fit_generator(generator=train_it, validation_data=val_it,epochs=300,
                    callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)

## load model from file

In [6]:
# model = load_model('ResNet50(bk)_19-12-11-12_38-150-1.00.hdf5')
# adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# model.compile(loss='binary_crossentropy',
#                   optimizer=adam_fine,
#                   metrics=['accuracy'])


# test Model


In [10]:

X_test,y_test,class_weights_test=create_sequence('datasets/FaceSwap/test/',frames_per_video_per_batch,frames_per_video)
test_it=DFVDSequence(X_test,y_test,test_video_per_batch,frames_per_video_per_batch)
# for i in range(0,test_it.__len__()):
#     temp=test_it.__getitem__(1)
#     print(temp[0].shape,temp[1].shape,test_it.dataset_size,test_it.frame_counter)
# # model.load_weights('2019-11-01_16.57.45_weights-improvement-01-1.00.hdf5')
# model.compile(loss='binary_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])
print(model.evaluate_generator(test_it))
test_it=DFVDSequence(X_test,y_test,test_video_per_batch,frames_per_video_per_batch)
y_prob=model.predict_generator(test_it)
y_classes = y_prob.argmax(axis=-1)
print(y_classes)
y_prob=model.predict_generator(test_it)
y_classes = y_prob.argmax(axis=-1)
print(y_classes)
# print('test loss:',loss)
# model=cl_basic(128,128,3,2)
# print_summary(model, line_length=150, positions=None, print_fn=None)
# model.compile(loss='binary_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])

In [ ]:
test_it=DFVDSequence(X_test,y_test,test_video_per_batch,frames_per_video_per_batch)
y_prob = model.evaluate_generator(test_it)
print(y_prob)
# y_classes = y_prob.argmax(axis=-1)
# print(y_classes)

In [ ]:

# model.fit_generator(generator=train_it, validation_data=val_it,epochs=150,initial_epoch=50,
                    callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)

In [11]:
# _id = dt.now().strftime("%y-%m-%d-%H_%M")
# print(_id)

In [7]:
# model.fit_generator(generator=train_it, validation_data=val_it,epochs=300,initial_epoch=150,
#                     callbacks=callbacks_list,shuffle=False,class_weight=class_weights_train,verbose=1)